In [ ]:
import matplotlib.pyplot as plt
from colicoords import save, CellPlot, CellListPlot, load, CellList, iter_subplots, IterCellPlot
import numpy as np
import os
from addict import Dict
import h5py
import multiprocessing as mp
from functools import partial
from functools import reduce
import operator
from matplotlib.lines import Line2D
import time
from dask.distributed import Client

In [ ]:
basedir = r'../src_data'

In [ ]:
client = Client('127.0.0.1:8786')
client


In [ ]:
t0 = time.time()
d = Dict()

# repeat 01
c41 = load(os.path.join(basedir, '20190930_yichen_deltaescv_c41_eyfp_escv_repeat_01', 'c41_with_spots.hdf5'))
epec = load(os.path.join(basedir, '20190930_yichen_deltaescv_c41_eyfp_escv_repeat_01', 'epec_with_spots.hdf5'))
d['20190930'].c41 = c41
d['20190930'].epec = epec
print('r1')

# repeat 02
c41 = load(os.path.join(basedir, '20191001_yichen_deltaescv_c41_eyfp_escv_repeat_02', 'c41_with_spots.hdf5'))
epec = load(os.path.join(basedir, '20191001_yichen_deltaescv_c41_eyfp_escv_repeat_02', 'epec_with_spots.hdf5'))
d['20191001'].c41 = c41
d['20191001'].epec = epec
print('r2')
# repeat 03
c41 = load(os.path.join(basedir, '20191002_yichen_deltaescv_c41_eyfp_escv_repeat_03', 'c41_with_spots.hdf5'))
epec = load(os.path.join(basedir, '20191002_yichen_deltaescv_c41_eyfp_escv_repeat_03', 'epec_with_spots.hdf5'))
d['20191002_03'].c41 = c41
d['20191002_03'].epec = epec
print('r3')
#repeat 04
c41 = load(os.path.join(basedir, '20191002_yichen_deltaescv_c41_eyfp_escv_repeat_04', 'c41_with_spots.hdf5'))
epec = load(os.path.join(basedir, '20191002_yichen_deltaescv_c41_eyfp_escv_repeat_04', 'epec_with_spots.hdf5'))
escc = load(os.path.join(basedir, '20191002_yichen_deltaescv_c41_eyfp_escv_repeat_04', 'escc_with_spots.hdf5'))
d['20191002_04'].c41 = c41
d['20191002_04'].epec = epec
d['20191002_04'].escc = escc
print('r4')

f = Dict()

# AHT 40
c41_02 = load(os.path.join(basedir, '20191004_c41_eyfp-escv_AHT40_repeats', 'c41_02_with_spots.hdf5'))
c41_03 = load(os.path.join(basedir, '20191004_c41_eyfp-escv_AHT40_repeats', 'c41_03_with_spots.hdf5'))
f.aht40['20191004_02'].c41 = c41_02
f.aht40['20191004_03'].c41 = c41_03
print('40')

# AHT 30
c41_02 = load(os.path.join(basedir, '20191005_c41_eyfp-escv_AHT30_repeats', 'c41_02_with_spots.hdf5'))
c41_03 = load(os.path.join(basedir, '20191005_c41_eyfp-escv_AHT30_repeats', 'c41_03_with_spots.hdf5'))
f.aht30['20191005_02'].c41 = c41_02
f.aht30['20191005_03'].c41 = c41_03
print('30')
f.aht20 = d

# AHT 30
c41_01 = load(os.path.join(basedir, '20191009_yichen_c41_eyfp-escv_AHT30_agarose_medium_repeats', 'c41_01_with_spots.hdf5'))
c41_02 = load(os.path.join(basedir, '20191009_yichen_c41_eyfp-escv_AHT30_agarose_medium_repeats', 'c41_02_with_spots.hdf5'))
f.aht30['20191009_01'].c41 = c41_01
f.aht30['20191009_02'].c41 = c41_02
print('30_1')

# AHT 25
c41_01 = load(os.path.join(basedir, '20191010_yichen_c41_eyfp-escv_AHT25_agarose_medium_repeats', 'c41_01_with_spots.hdf5'))
c41_02 = load(os.path.join(basedir, '20191010_yichen_c41_eyfp-escv_AHT25_agarose_medium_repeats', 'c41_02_with_spots.hdf5'))
c41_03 = load(os.path.join(basedir, '20191010_yichen_c41_eyfp-escv_AHT25_agarose_medium_repeats', 'c41_03_with_spots.hdf5'))
f.aht25['20191010_01'].c41 = c41_01
f.aht25['20191010_02'].c41 = c41_02
f.aht25['20191010_03'].c41 = c41_03

#AHT 20
epec_aht20 = load(os.path.join(basedir, '20191022_deltaescv_eyfp-escv_aht20', 'epec_yfp-escv_aht20_with_spots.hdf5'))
f.aht30['20191022'].epec = epec_aht20  #EPEC!!!

#AHT 25/20 RSTU
c41_aht20 = load(os.path.join(basedir, '20191023_c41_eyfp-escv_escrstu_AHT25', 'c41_yfp-escv_aht20with_spots.hdf5'))
f.aht20['20191023'].c41 = c41_aht20

c41_aht25 = load(os.path.join(basedir, '20191023_c41_eyfp-escv_escrstu_AHT25', 'c41_yfp-escv_aht25with_spots.hdf5'))
f.aht25['20191023'].c41 = c41_aht25

t1 =  time.time()

print(t1 - t0)

f

In [ ]:
def zscore(d):
    return (d - d.mean()) / d.std()

def filter_cells(cells, cell_type, area_min=350, chisq_max=0.001):
    cells = cells.copy()

    #filter by radius
    cells.measure_r(mode='mid', data_name='brightfield')
    if cell_type=='c41':
        rmin=3.5041735933766798
        rmax=5.46023075024
    elif cell_type=='epec':
        rmin=5.83941563572067
        rmax=7.102502047477
    else:
        print('EMPTY')
        return CellList([])

    r = cells.radius
    bmin = r > rmin
    bmax = r < rmax
    b = np.logical_and(bmin, bmax)

    print('Removed {} by radius'.format(np.sum(~b)))
    cells = cells[b]

    #filter by area
    b = cells.area > area_min
    print('Removed {} by area'.format(np.sum(~b)))
    cells = cells[b]

    #filter by brightfield radial distribution
    xv, yv = cells.r_dist(3, 0.2, data_name='brightfield', norm_x=True)
    means = np.mean(yv, axis=1)
    normed = yv / means[:, np.newaxis]

    overall_mean = normed.mean(axis=0)
    chisq = np.mean((overall_mean - normed)**2, axis=1)

    b = chisq < chisq_max
    print('Removed {} by brightfield shape'.format(np.sum(~b)))

    means = np.mean(yv[b], axis=1)
    normed = yv[b] / means[:, np.newaxis]

    overall_mean = normed.mean(axis=0)
    chisq = np.mean((overall_mean - normed)**2, axis=1)

    b1 = chisq < chisq_max
    print('Removed {} by brightfield shape 1'.format(np.sum(~b1)))

    cells = cells[b][b1]

    _int = cells.get_intensity(data_name='g500')
    b = _int < 50000
    cells = cells[b]
    print('Removed {} by intensity'.format(np.sum(~b)))

    num_pix_70k = np.array([np.sum(c.data.data_dict['g500'] > 70000) for c in cells])
    b = num_pix_70k < 50
    nc = cells[~b]
    cells = cells[b]
    print('Removed {} by pixels intensity'.format(np.sum(~b)))

    return cells


In [ ]:
def worker(cells, k, dst):
    filtered = filter_cells(cells, k)
    save(dst + '.hdf5', cells)
    save(dst + '_filtered.hdf5', filtered)

    return filtered

In [ ]:
def r_dict(d, d_out, path):
    pth = os.path.join(*path)
    if not os.path.exists(pth):
        os.mkdir(pth)
    for k, v in d.items():
        if isinstance(v, Dict):
            r_dict(v, d_out, path + [k])
        else:
            print(path, k)
            dst = os.path.join(pth, k + '.hdf5')
            save(dst, v)

            filtered = filter_cells(v, k)
            parent = reduce(operator.getitem, path[1:], d_out)
            parent[k + '_filtered'] = filtered
            dst_f = os.path.join(pth, k + '_filtered.hdf5')
            save(dst_f, filtered)

In [ ]:
def r_dict_save(d, path):
    pth = os.path.join(*path)
    if not os.path.exists(pth):
        os.mkdir(pth)
    for k, v in d.items():
        if isinstance(v, Dict):
            yield from r_dict_save(v, path + [k])
        else:
            print(path, k)
            dst = os.path.join(pth, k)
            c= client.submit(worker, v, k, dst)
            yield c

In [ ]:
r_dict_save(f, ['root_20191213'])
